In [48]:
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'/Users/ryan/github/prosodic')
sys.path.insert(0,'/Users/ryan/github/lltk')
from llmdh import *
from sqlitedict import SqliteDict
import os

In [49]:
corproot='/Users/ryan/github/bechdeltest/data/corpus'
dfmeta = pd.read_csv(corproot+'/metadata.csv')
dfmeta['release_date']=pd.to_numeric(dfmeta['release_date'], errors='coerce')
dfmeta['dec']=dfmeta['release_date']//10*10
n=100
ids=pd.concat(gdf if len(gdf)<=n else gdf.sample(n) for g,gdf in dfmeta.query('1940<=release_date').groupby('dec')).id.sample(frac=1)
# ids = dfmeta.query('1940<=release_date').id.sample(frac=1)
len(ids)

659

In [50]:
def tostr(x):
    try:
        return str(int(x))
    except:
        return x
dfgenre=pd.read_pickle('data.bechdel.dfall.pkl')
dfgenre['imdb_id']=dfgenre.imdb_id.apply(tostr)
imdb2id = dict(zip(dfgenre.index, dfgenre.imdb_id))
# len(imdb2id)

In [51]:
# dbgenre=SqliteDict('data.imdb_genre.db', flag='r')
ids=[x for x in ids if x not in imdb2id]
len(ids)

398

In [52]:


# df.sort_values('tmdb_release_date',ascending=False).head(10)
def iter_scenes(id):
    fn=os.path.join(corproot,'texts',id,f'{id}.dialogue.csv')
    if os.path.exists(fn):
        df=pd.read_csv(fn).fillna('')

        o=[]
        for g,gdf in df.groupby('scene_num'):
            scene=[]
            for i,row in gdf.iterrows():
                if not scene and row.scene_desc: scene.append(f'\t\t[{row.scene_desc}]')
                if row.narration:
                    scene.append(f'\t\t({row.narration})')
                line=row.speaker
                if row.direction: line+=f' ({row.direction})'
                line+=f': {row.speech}'
                scene.append(line)
            yield g,('\n\n'.join(scene))

    # story=next(iter_scenes('10-Things-I-Hate-About-You'))

In [53]:
system_prompt='''You are a literary critic assessing whether scenes from a screenplay pass the "Bechdel test", which involves 3 components: 
1. Does the scene contain more than 1 female character?
2. If (1) is true, do at least two female characters speak to one another?
3. If (2) is true, do they speak about something other than a man?

Read the following story and return valid JSON dictionary containing this information.
'''

example_prompts=[
    (
'''
        [INT. HOUSE]

EMILY: I don't know, Emma, what do you think?

EMMA: I don't know either! Michael seems too risky to date to me.

EMILY: But how do you know?

EMMA: Well, I—

        (Just then MICHAEL enters through the back door.)

MICHAEL: What are you guys talking about?

        (EMILY and EMMA blush.)

EMMA: Nothing!

EMILY: Well, almost nothing.
''',
'''
{
    "female_characters": ["Emily", "Emma"],
    "male_characters": ["Michael"],
    "female_characters_speak_to_each_other": true,
    "female_characters_speak_about_something_other_than_a_man": false,
    "explanation": "Emily and Emma, both women, speak to each other, but their conversation is limited that regarding the reputation of Michael, a man."
}
'''
    )
]


In [54]:
dbfn=os.path.join(PATH_DATA,'data.bechdel.db')
db = SqliteDict(dbfn, autocommit=True)

In [64]:
k=next(db.keys())
k
for g,scene in iter_scenes(k):
    if g==15:
        break

In [66]:
print(scene)

		[INT. KENNY'S THAI FOOD DINER - DAY]

		(Kat and Mandella pick apart their pad thai. Mandella is smoking.)

KAT: So he has this huge raging fit about Sarah Lawrence and insists that I go to his male-dominated, puking frat boy, number one golf team school. I have no say at all.

MANDELLA: William would never have gone to a state school.

KAT: William didn't even go to high school

MANDELLA: That's never been proven

KAT: Neither has his heterosexuality.

		(Mandella replies with a look of ice. Kat uses the moment to stub out Mandella's cigarette.)

KAT: I appreciate your efforts toward a speedy death, but I'm consuming. Do you mind?

MANDELLA (continuing pointing at her food): Does it matter?

KAT: If I was Bianca, it would be, "Any school you want, precious. Don't forget your tiara."

		(They both look up as Patrick enters. He walks up to the counter to place his order. Mandella leans toward Kat with the glow of fresh gossip)

MANDELLA: Janice Parker told me he was a roadie for Maril

In [56]:
id='10-Things-I-Hate-About-You'

def parse_script(id,model='gemini-pro', force=False, progress=True):
    if not force and id in db and type(db[id])==pd.DataFrame: return db[id]

    o=[]
    for scene_num,scene in tqdm(list(iter_scenes(id)),position=0, disable=not progress):
        res=LLM.generate(
            scene,
            system_prompt=system_prompt,
            example_prompts=example_prompts,
            model=model,
            verbose=False
        )
        try:
            resd=json.loads('{' + res.split('{',1)[-1].split('}')[0] + '}')
            resd['scene_num']=scene_num
            o.append(resd)
        except Exception as e:
            # print('!!',e)
            pass
        # break
    try:
        df=pd.DataFrame(o).fillna('')
        df['num_female_characters']=df['female_characters'].apply(lambda x: len(x) if x else 0)
        df['num_male_characters']=df['male_characters'].apply(lambda x: len(x) if x else 0)
        for k in ['female_characters_speak_to_each_other', 'female_characters_speak_about_something_other_than_a_man']:
            df[k]=df[k].apply(lambda x: x==True)
    except Exception as e:
        # print('!!!!',e)
        pass
        df = pd.DataFrame()
        
    db[id] = df
    return df

In [57]:
# pd.options.display.max_rows=100
# keys=list(db.keys())
# sumdf=pd.DataFrame([db[k].mean(numeric_only=True) for k in keys], index=keys).rename_axis('id').dropna()
# sumdf.sort_values('female_characters_speak_about_something_other_than_a_man')
# # sumdf.sort_values('female_characters_speak_to_each_other')

In [58]:
def parse_all_scripts(ids, model='gemini-pro'):
    for id in tqdm(ids,position=0):
        parse_script(id,progress=True, model=model)



In [59]:
parse_all_scripts(ids, model='gemini-pro')

100%|██████████| 28/28 [01:24<00:00,  3.02s/it]
0it [00:00, ?it/s]0/398 [33:22<16:23:00, 152.01s/it]
100%|██████████| 78/78 [03:39<00:00,  2.81s/it]it]
0it [00:00, ?it/s]5/398 [57:28<9:51:29, 95.15s/it]
100%|██████████| 50/50 [02:30<00:00,  3.00s/it]11s/it]
0it [00:00, ?it/s]5/398 [3:23:43<14:01:07, 156.25s/it]
 20%|██        | 80/398 [4:03:44<16:08:54, 182.81s/it]


KeyboardInterrupt: 

E0421 14:45:41.240238000 11071418368 ssl_transport_security.cc:1519]   Handshake failed with fatal error SSL_ERROR_SSL: error:10000410:SSL routines:OPENSSL_internal:SSLV3_ALERT_HANDSHAKE_FAILURE.
E0421 14:45:51.264949000 6238580736 ssl_transport_security.cc:1519]    Handshake failed with fatal error SSL_ERROR_SSL: error:10000410:SSL routines:OPENSSL_internal:SSLV3_ALERT_HANDSHAKE_FAILURE.
